# Regresión Logística con Python

Para esta sesión, estaremos trabajando con el [Conjunto de datos Titanic de Kaggle](https://www.kaggle.com/c/titanic). Este es un conjunto de datos muy famoso y a menudo se utiliza para introducir a estudiantes al Aprendizaje de Máquina (Machine Learning)! 

Trataremos de predecir una clasificación - supervivencia o fallecimiento.
Iniciemos nuestra comprensión la de la implementación de la Regresión Logística con Python para clasificar.

Utilizaremos una versión "semi-limpia" del conjunto de datos Titanic.  NOTA:  Si decide utilizar el conjunto de datos directamente de Kaggle, puede que necesite hacer más limpieza de la que estaremos haciendo acá.


## Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Los datos

In [ ]:
train = pd.read_csv('titanic_train.csv')

In [ ]:
train.head()

# Análisis Exploratorio de Datos (EDA)

Empezaremos con el chequeo por datos faltantes!

## Datos faltantes

Hemos visto varias formas de hacerlo, para algo diferente, utilizaremos **Seaborn** para crear un mapa de calor que nos muestre dónde hacen falta datos!

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

Aproximadamente el 20% de los datos de **Edad** hacen falta.  Esta proporción probablemente es lo suficientemente pequeño como para llenar valores con alguna forma de imputación.  Si vemos la columna **Cabin**, parece que simplemente hay demasiados datos faltantes como para hacer algo útil, al menos a un nivel básico.  Lo más probable es que botaremos esta columna, o cambiamos el significado a algo como "Cabina conocida:  1 o 0".

Continuemos con más visualizaciones.

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')

In [ ]:
sns.displot(train['Age'].dropna(),color='darkred',bins=30)

In [ ]:
train['Age'].hist(bins=30,color='darkred',alpha=0.7)

In [ ]:
sns.countplot(x='SibSp',data=train)

In [ ]:
train['Fare'].hist(color='green',bins=40,figsize=(8,4))

____
### Cufflinks para gráficas
___
 Tomemos un momento breve para ve algo de **Cufflinks**!

In [ ]:
import cufflinks as cf
cf.go_offline()

In [ ]:
train['Fare'].iplot(kind='hist',bins=30,color='green')

___
## Limpieza de datos

Deseamos dar valores a los datos faltante de edad, en vez de solo eliminar las filas que no tengan ese valor.  Una forma de hacerlo es usando la edad promedio de todos los pasajeros (imputación)

Sin embargo, vamos a ser un poco más inteligentes y usaremos la edad promedio por Clase en la que viajan los pasajeros.  Por ejemplo:


In [ ]:
plt.figure(figsize=(12, 7))
sns.boxplot(x='Pclass',y='Age',data=train,palette='winter')

Podemos ver que los pasajeros más "pudientes" en las clases más altas, tienden a ser mayores, que hace mucho sentido.

Utilizaremos estos valores promedio de edad basados en **Pclass** para la edad.

In [ ]:
def imputar_edad(cols):
    edad = cols[0]
    clase = cols[1]
    
    if pd.isnull(edad):

        if clase == 1:
            return 37

        elif clase == 2:
            return 29

        else:
            return 24

    else:
        return edad

Ahora apliquemos esa función!

In [ ]:
train['Age'] = train[['Age','Pclass']].apply(imputar_edad,axis=1)

Veamos el mapa de calor de nuevo!

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

Fabuloso! Eliminemos la columna **Cabin** y la fila en **Embarked** que tiene un NaN.

In [ ]:
train.drop('Cabin',axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
train.dropna(inplace=True)

## Convertir Columnas Categóricas

Será necesario convertir las columnas categóricas a variables "dummy" o comodines utilizando Pandas!  De no hacerlo, nuestro algortimo de Machine Learning no podrá tomar esas columnas como entradas ya que necesita que sean numéricas.



In [ ]:
train.info()

In [ ]:
sexo = pd.get_dummies(train['Sex'],drop_first=True)
embarque = pd.get_dummies(train['Embarked'],drop_first=True)

In [ ]:
train.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [ ]:
train = pd.concat([train,sexo,embarque],axis=1)

In [ ]:
train.head()

Ahora sí, nuestros datos están listos para modelar!

# Preparar un modelo de Regresión Logística

Iniciemos con dividir nuestros datos en un cojunto para entrenar (training) y otro para probar (test)...hay otro archivo **test.csv** con el que puede hacer pruebas si acaso desea usar todo el conjunto inicial para entrenar.

## Partir en Train / Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1), 
                                                    train['Survived'], test_size=0.30, 
                                                    random_state=101)

## Entrenamiento y Predicción

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression(max_iter = 500)
logmodel.fit(X_train,y_train)

In [ ]:
predictions = logmodel.predict(X_test)

Pasémos a evaluar el modelo!

## Evaluación

Podemos chequear: precision, recall y f1-score utilizando **classificationn report**!

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,predictions))

No está mal!  Puede que desee explorar otras formas de organizar los datos (feature engineering) y el otro archivo **titanic_text-csv**, algunas sugerencias de trabajar los datos:

* Intente extraer el título (Dr., Mr., Mrs, etc.) de el nombre, y dejarlo como otra columna
* Quizás la letra de la Cabina podría ser una variable importante
* ¿Hay alguna información de el **Ticket** que podría ser útil?